# Docking a set of smiles with AutoDock Vina

Ryan L. Melvin

23 May 2016
Updated: October 7, 2016

Docking software allows for quick (compared to molecular dynamics docking simulation) estimates of complexes of a receptor and ligand (e.g., protein and drug-link compound). To  identify potential candidates for drug discovery, ligand libraries must be used. One such examples is ZINC[1-2] Once a library or subset thereof has been selected, docking software such AutoDock Vina[3] can provide a quick, efficient calculation of binding sites and affinities.

In [ ]:
# Python modules needed:
import subprocess
from shutil import move
import os
import re

Who are you (on DEAC cluster) and how many simultaneous submissions are ok?

In [ ]:
deac_user = 'melvrl13'
max_simultaneous_submissions = 500

First, select the library (or subset thereof) and provide it as a set of smiles strings

In [ ]:
# Assuming you have the library to be docked in smiles strings
library = '23_t90.smi'

You'll need a receptor. 

In [ ]:
receptor = 'receptor.pdb'

Let's count how many drug-like compounds are in the provided library

In [ ]:
with open(library, 'r+') as smiles_strings:
    num_drugs = sum(1 for line in smiles_strings)

You'll need to adjust the docking helper script with your account info, the location of babel and MGLTools, and the appropriate awk command for what you want stored. I have an example script at the end of this notebook.

In [ ]:
docking_helper = 'docking.slurm'

Now, we're going to have to do a bunch of once. The following script assumes you're using a distributed computer environment with slurm. I use a helper file that takes care of all the slurm details, whose code I provide at the end of this document. I'll also give an example smiles string at the end.

In [ ]:
squeue_command = 'squeue -u ' + deac_user
cwd = os.getcwd()
with open(zinc_library, 'r+') as smiles_strings:
    smiles_text = smiles_strings.read()
    drugs = re.findall('(ZINC\d+)', smiles_text)
for drug_num in range(1, num_drugs + 1):
    squeue = subprocess.Popen(squeue_command, shell=True, stdout=subprocess.PIPE)
    out, err = squeue.communicate()
    out = out.splitlines()
    count = sum (1 for line in out)
    while count > max_simultaneous_submissions:
        time.sleep(5)
        squeue = subprocess.Popen(squeue_command, shell=True, stdout=subprocess.PIPE)
        out, err = squeue.communicate()
        out = out.splitlines()
        count = sum (1 for line in out)
    compound = drugs[drug_num - 1]
    compound_dir = os.path.join(cwd, compound)
    if not os.path.exists(compound_dir):
        os.makedirs(compound_dir)
    submit_docking_command = (
            'sbatch --output=/dev/null --export=o=' +  compound + ',x=' + str(drug_num) + ',r=' + receptor + ',l=' ' + docking_helper
            )
    docking_job = subprocess.Popen(submit_docking_command, shell=True, cwd=compound_dir, stdout=subprocess.PIPE)

# Analysis
Now that you have the results, let's analyze them. You'll need to adjust paths to fit what you're doing. I put all of my enery results in a file using awk and then replaced the spaces with commas. Something like
```bash
for filename in *.dat; do
awk '{if (NF==3) print $1,$2,FILENAME}' ${filename} >> energies_awk.txt
done
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

I compressed my results using pandask pickle command so that they take up less space and can easily be read into python for future analysis.

In [ ]:
df = pandas.read_csv('energies_csv.txt', index_col=False)
df.to_pickle('result.pk')

Thigs get even more specific here. I was comparing energies for two binding sites. What follows is intended as an example, not a programatic process for everyone to use every time. 

In [ ]:
df = pd.read_pickle('result.pk') # load

In [ ]:
# Look at the 'F10' binding site
df[df.site=='F10'].sort_values(by='energy').head() # top results

In [ ]:
df[df.site=='F10'].energy.hist() # histogram all results
plt.title('F10 site')
plt.xlabel('energy')
plt.ylabel('count')

In [ ]:
# Look at the 'Acid' binding site
df[df.site=='Acid'].sort_values(by='energy').head() # top results

In [ ]:
df[df.site=='Acid'].energy.hist() # histogram all results
plt.title('Acid site')
plt.xlabel('energy')
plt.ylabel('count')

In [ ]:
# Comparison
top = df[df.model==1]
F = top[top.site=='F10']
F = F.drop(['site', 'model'] , 1)
A = top[top.site=='Acid']
A = A.drop(['site', 'model'], 1)
r = F.merge(A, on='drug', suffixes=('_F10', '_Acid'))
r = r.set_index('drug')
r.head()

In [ ]:
# Searching for a site that met a range of energies criteria
F10_lim = -7.0 # Change this
Acid_lim = -6.0 # And/or this
r[(r.energy_F10 < F10_lim) & (r.energy_Acid > Acid_lim)]

## Citations
[1] Irwin JJ, Shoichet BK (2005) ZINC - a free database of commercially available com- pounds for virtual screening. J Chem Inf Model 36:177–182. doi:10.1002/chin. 200516215

[2] Irwin JJ, Sterling T, Mysinger MM et al (2012) ZINC: a free tool to discover chemis- try for biology. J Chem Inf Model 52:1757– 1768. doi:10.1021/ci3001277

[3] Trott O, Olson AJ (2010) Software news and update AutoDock Vina: improving the speed and accuracy of docking with a new scoring function, efficient optimization, and multi- threading. J Comput Chem 31:455–461. doi:10.1002/jcc

### docking_submit.slurm
```bash
#!/bin/bash -l
#SBATCH --partition=medium
#SBATCH --account=salsburyGrp
#SBATCH --nodes=1
#SBATCH --mail-user=melvrl13@wfu.edu
#SBATCH --tasks-per-node=8
#SBATCH --mem=16gb
#SBATCH --time=01:00:00

module load vina/1.1.2-intel-2012

/home/salsbufr/babel/bin/babel -ismi ${l} --gen3d -f ${x} -l ${x} -opdb zinc.pdb

#/home/salsbufr/babel/bin/obminimize zinc${x}.pdb

/home/luy/MGLTools-1.5.4/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py -l zinc.pdb

vina --receptor ${r} --exhaustiveness 99 --ligand zinc.pdbqt  --center_x 23.7 --center_y 3.6  --center_z -1.2 --size_x 20 --size_y 20 --size_z 20 --out out.pdbqt > out.log

#If you're doing something comparative
#vina --receptor ${r} --exhaustiveness 99 --ligand zinc.pdbqt  --center_x 33.6 --center_y -8.3 --center_z 22.6 --size_x 28 --size_y 28 --size_z 28 --out F10.pdbqt > F10.log

# I wanted a certain range of energies. You should adjust the awk command for what you want and where you want it stored.
awk '{if (NF==4 && $2<10.0) print $2,$1,FILENAME}' out.log > ../energies/out_${o}.dat

#awk '{if (NF==4 && $2<10.0) print $2,$1,FILENAME}' F10.log > ../energies/F10_${o}.dat

module unload vina/1.1.2-intel-2012

rm -r ../${o}
```

### Smiles example
Cc1cc(no1)NC(=O)CCn2cnc3c(c2=O)cnn3C	ZINC54722086